# ItemItem Result Analysis

In [1]:
import pandas as pd
import numpy as np

## Compare Predictions

In [27]:
sample = pd.read_csv('build/pairs-ml-100k.csv', names=['user', 'item'])
sample.head()

,user,item
0,147,274
1,147,302
2,147,710
3,147,890
4,147,182


In [28]:
java = pd.read_csv('build/lenskit/item-item-ml-100k-preds.csv')
java = pd.merge(sample, java, how='outer')
java.head()

,user,item,prediction
0,147,274,3.476850
1,147,302,4.610622
2,147,710,3.483527
3,147,890,1.770824
4,147,182,3.765012


In [19]:
lkpy = pd.read_csv('build/lkpy/item-item-ml-100k-preds.csv')
lkpy.head()

,user,item,prediction
0,2,967,3.244902
1,2,1671,NaN
2,2,1467,NaN
3,2,1544,NaN
4,2,649,NaN


In [29]:
merged = pd.merge(java.rename(columns={'prediction': 'java'}), lkpy.rename(columns={'prediction': 'lkpy'}))
merged.head()

,user,item,java,lkpy
0,147,274,3.476850,3.848578
1,147,302,4.610622,4.710038
2,147,710,3.483527,4.053542
3,147,890,1.770824,1.980225
4,147,182,3.765012,4.047028


In [30]:
merged['diff'] = merged.java - merged.lkpy
merged['diff'].describe()

count    4.940000e+02
mean    -2.842731e-02
std      1.283738e-01
min     -1.115043e+00
25%     -4.440892e-16
50%      0.000000e+00
75%      0.000000e+00
max      8.201200e-01
Name: diff, dtype: float64

In [12]:
merged.sort_values(by='diff', ascending=False).head(10)

,user,item,java,lkpy,diff
1014,510,666,3.318433,2.498313,0.820120
162,813,822,3.032375,2.754238,0.278136
279,571,148,3.116507,2.857219,0.259288
164,813,484,3.686964,3.434018,0.252946
278,571,1266,3.137460,2.895693,0.241768
854,182,1230,1.494465,1.279321,0.215144
442,88,1489,1.093545,0.984959,0.108586
277,571,647,3.937254,3.835087,0.102166
1005,510,84,2.737743,2.682477,0.055266
611,906,1412,1.528032,1.479457,0.048575
